In [ ]:
%pip install torch transformers datasets accelerate bitsandbytes trl bert_score peft evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 155.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 65.7 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 104.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 184.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 196.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 162.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 182.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.4 MB/s eta 0:00:00

[notice] A new release of pip 

In [2]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. 사용할 모델의 Hugging Face ID 확인
# LG AI Research의 EXAONE 4.0 모델 ID는 다음과 같습니다.
# 이 ID를 사용하여 모델을 로드합니다.
model_id = "LGAI-EXAONE/EXAONE-4.0-1.2B"

# 2. 모델과 토크나이저를 로드 (인터넷에서 다운로드)
# trust_remote_code=True는 특정 모델 아키텍처(custom code)를 로드할 때 필요할 수 있습니다.
# EXAONE 4.0 모델은 이 옵션이 필요합니다.
print(f"⏳ Hugging Face에서 '{model_id}' 모델 로드 중...")
try:
    model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    print("✅ 모델과 토크나이저가 성공적으로 로드되었습니다.")
except Exception as e:
    print(f"❌ 모델 로드 중 오류 발생: {e}")
    print("Hugging Face Hub에서 모델 ID를 다시 확인하거나, 인터넷 연결을 확인해주세요.")
    print("transformers 라이브러리 버전이 최신인지 확인해주세요 (pip install --upgrade transformers).")
    exit() # 오류 발생 시 스크립트 종료

# 3. 로컬에 저장할 폴더 경로 정의
# 이 폴더에 모델 가중치, 설정 파일, 토크나이저 파일 등 모든 것이 저장됩니다.
# 이 폴더 이름을 나중에 Django settings.py에 사용할 것입니다.
local_save_directory = "./downloaded_exaone_4.0_1.2b"

# 4. 폴더가 없다면 생성
if not os.path.exists(local_save_directory):
    os.makedirs(local_save_directory)

# 5. 모델과 토크나이저를 로컬 폴더에 저장
print(f"⏳ 모델과 토크나이저를 '{local_save_directory}'에 저장 중...")
try:
    model.save_pretrained(local_save_directory)
    tokenizer.save_pretrained(local_save_directory)
    print(f"✅ 모델과 토크나이저가 '{local_save_directory}'에 성공적으로 저장되었습니다.")
    print("이 폴더 안에는 모델 가중치(.bin 또는 .safetensors), config.json, tokenizer.json 등 모든 필수 파일이 포함됩니다.")
except Exception as e:
    print(f"❌ 모델 저장 중 오류 발생: {e}")

# 6. 저장된 모델을 다시 로드하여 확인 (선택 사항)
print(f"\n💡 저장된 모델을 '{local_save_directory}'에서 다시 로드하여 확인...")
try:
    loaded_model = AutoModelForCausalLM.from_pretrained(local_save_directory, trust_remote_code=True)
    loaded_tokenizer = AutoTokenizer.from_pretrained(local_save_directory, trust_remote_code=True)
    print("✅ 로컬에 저장된 모델이 성공적으로 다시 로드되었습니다.")
    # 간단한 테스트 (예: 토크나이저 크기 확인)
    print(f"Loaded tokenizer vocab size: {len(loaded_tokenizer)}")
except Exception as e:
    print(f"❌ 로컬 저장된 모델 로드 중 오류 발생: {e}")

⏳ Hugging Face에서 'LGAI-EXAONE/EXAONE-4.0-1.2B' 모델 로드 중...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

✅ 모델과 토크나이저가 성공적으로 로드되었습니다.
⏳ 모델과 토크나이저를 './downloaded_exaone_4.0_1.2b'에 저장 중...
✅ 모델과 토크나이저가 './downloaded_exaone_4.0_1.2b'에 성공적으로 저장되었습니다.
이 폴더 안에는 모델 가중치(.bin 또는 .safetensors), config.json, tokenizer.json 등 모든 필수 파일이 포함됩니다.

💡 저장된 모델을 './downloaded_exaone_4.0_1.2b'에서 다시 로드하여 확인...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로컬에 저장된 모델이 성공적으로 다시 로드되었습니다.
Loaded tokenizer vocab size: 102400
